In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

In [ ]:
# Connection to a local Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Initiate session
session = cluster.connect()

In [ ]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [ ]:
# Query 1: Creating a table to provide the artist, song title and song's length in the music app history that was heard 
# during sessionId = 338, and itemInSession = 4
# The query wants to find the song listend to at session 338 and item 4. It makes the most sense to partition by each \
# session so that all rows in a single session can be in the same node and therefore easily retrievable \
# Then to make each row unique use a cluster column of iteminsession, which will also order the sessions correctly.

query = "CREATE TABLE IF NOT EXISTS music_app_session_history"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_session_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assigning which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_app_session_history WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print (row.artist, row.song, row.length)

data = []

for row in rows:
    data.append({'artist':row.artist, 'song':row.song, 'length':row.length})

df = pd.DataFrame(data)

df

In [ ]:
# Query 2: Creating a table to provide the name of artist, song (sorted by itemInSession) and user (first and last name) \
# for userid = 10, sessionid = 182
# The query wants to find the songs that user 10 listened to in session 182 in order by iteminsession. Therefore \
# partioning by userid and SessionId makes sense so that all rows for each user and session can be in the same node and \
# easily retrievable. 
# iteminsession is the first clustering column to order the songs correctly 

query = "CREATE TABLE IF NOT EXISTS user_session_song_history"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_song_history (userId, sessionId, itemInSession, artist, song, \
                                                        firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assigning which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName FROM user_session_song_history WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print (row.artist, row.song, row.firstname, row.lastname)

data2 = []

for row in rows:
    data2.append({'artist':row.artist, 'song':row.song, 'firstName':row.firstname, 'lastName':row.lastname})

df2 = pd.DataFrame(data2)

df2

In [ ]:
# Query 3: Creating a table to provide every user name (first and last) in my music app history who listened to \
# the song 'All Hands Against His Own'
# The query wants find all users that have listened to a specific song. Therefore the partion key song makes sense \
# to store all rows with a specific song on each node, making rows with 'All Hands Against His Own' easily retrievable. \
# then the clustering column userId is used, without it only one user would be returned as there would be no way to \
# differentiate the users who listened to the song

query = "CREATE TABLE IF NOT EXISTS song_user_history"
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [ ]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assigning which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
# SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName FROM song_user_history \
         WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#for row in rows:
#    print (row.firstname, row.lastname)

data3 = []

for row in rows:
    data3.append({'firstName':row.firstname, 'lastName':row.lastname})

df3 = pd.DataFrame(data3)

df3

In [ ]:
# Drop the tables before closing out the sessions
query = "drop table music_app_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Closing the session and the cluster
session.shutdown()
cluster.shutdown()